In [2]:
# !pip install boto3
# !pip install s3fs

In [3]:
import boto3
import pandas as pd
import configparser
import s3fs
config = configparser.ConfigParser()

config.read('aws.cfg')
AWS_ACCESS_KEY = config['AWS']['aws_access_key']
AWS_SECRET_KEY = config['AWS']['aws_secret_key']

In [4]:
bucket_name = 'capstone-techcatalyst-raw'
s3_client = boto3.client('s3', aws_access_key_id = AWS_ACCESS_KEY, aws_secret_access_key = AWS_SECRET_KEY)

In [5]:
green_taxi_obj = s3_client.list_objects_v2(Bucket=bucket_name, Prefix='green_taxi')
green_taxi_files = [obj['Key'] for obj in green_taxi_obj['Contents']]
green_taxi_files.pop(0)

'green_taxi/'

In [6]:
yellow_taxi_obj = s3_client.list_objects_v2(Bucket=bucket_name, Prefix='yellow_taxi')
yellow_taxi_files = [obj['Key'] for obj in yellow_taxi_obj['Contents']]
yellow_taxi_files.pop(0)

'yellow_taxi/'

In [15]:
yellow_taxi_2023_files = yellow_taxi_files[:4]
yellow_taxi_2024_files = yellow_taxi_files[4:]

['yellow_taxi/yellow_tripdata_2024-01.parquet',
 'yellow_taxi/yellow_tripdata_2024-02.parquet',
 'yellow_taxi/yellow_tripdata_2024-03.parquet',
 'yellow_taxi/yellow_tripdata_2024-04.parquet',
 'yellow_taxi/yellow_tripdata_2024-05.parquet']

In [16]:
def generate_url(bucket, file):
    url = f's3://{bucket}/{file}'
    return url

In [13]:
green_taxi = pd.DataFrame()
for file in green_taxi_files:
    url = generate_url(bucket_name, file)
    temp = pd.read_parquet(url, storage_options={'key' : AWS_ACCESS_KEY, 'secret' : AWS_SECRET_KEY})
    # print(f"Adding {temp.shape[0]} records from {file}")
    green_taxi = pd.concat([green_taxi, temp], axis=0)

In [17]:
yellow_taxi_2023 = pd.DataFrame()
for file in yellow_taxi_2023_files:
    url = generate_url(bucket_name, file)
    temp = pd.read_parquet(url, storage_options={'key' : AWS_ACCESS_KEY, 'secret' : AWS_SECRET_KEY})
    # print(f"Adding {temp.shape[0]} records from {file}")
    yellow_taxi_2023 = pd.concat([yellow_taxi_2023, temp], axis=0)

: 

In [ ]:
print(f"Number of records: {green_taxi.shape[0]}")

Number of records: 544947


In [ ]:
green_taxi.columns

Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge'],
      dtype='object')